# Training on 1,2 testing on 3,4,5,6

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load the dataset
data = pd.read_csv("final_dataset.csv")

# Filter data for training with fitzpatrick_scale values 1 and 2
train_data = data[data['fitzpatrick_scale'].isin([1, 2])]

# Filter data for testing with fitzpatrick_scale values 3, 4, 5, and 6
test_data = data[data['fitzpatrick_scale'].isin([3, 4, 5, 6])]

# Define the image data generator with augmentation (optional)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Define the input image size and batch size
img_size = (224, 224)
batch_size = 32

# Create the generators for training and testing data
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory=None,
    x_col="local_filename",
    y_col="label",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_data,
    directory=None,
    x_col="local_filename",
    y_col="label",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False,
)

# Define the InceptionNet model and load pre-trained weights
base_model = InceptionV3(weights='imagenet', include_top=False)

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully-connected layer with 1024 neurons
x = Dense(1024, activation='relu')(x)

# Add a final output layer with one neuron per class
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze all layers in the base model for transfer learning
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with categorical cross-entropy loss and Adam optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on the training data
history = model.fit(train_generator, epochs=20, validation_data=test_generator)

# Evaluate the model on the testing data
test_loss, test_acc = model.evaluate(test_generator)
print("Test accuracy:", test_acc)


Found 6913 validated image filenames belonging to 114 classes.
Found 5302 validated image filenames belonging to 114 classes.
Epoch 1/20
217/217 [==============================] - 835s 4s/step - loss: 4.2662 - accuracy: 0.0891 - val_loss: 4.3619 - val_accuracy: 0.0764
Epoch 2/20
217/217 [==============================] - 795s 4s/step - loss: 3.7161 - accuracy: 0.1444 - val_loss: 4.2530 - val_accuracy: 0.0990
Epoch 3/20
217/217 [==============================] - 825s 4s/step - loss: 3.4642 - accuracy: 0.1813 - val_loss: 4.1082 - val_accuracy: 0.1101
Epoch 4/20
217/217 [==============================] - 803s 4s/step - loss: 3.2599 - accuracy: 0.2190 - val_loss: 4.0244 - val_accuracy: 0.1149
Epoch 5/20
217/217 [==============================] - 791s 4s/step - loss: 3.0971 - accuracy: 0.2465 - val_loss: 4.0941 - val_accuracy: 0.1190
Epoch 6/20
217/217 [==============================] - 832s 4s/step - loss: 2.9436 - accuracy: 0.2730 - val_loss: 4.1951 - val_accuracy: 0.1245
Epoch 7/20
217/2

NameError: name 'accuracy' is not defined

In [2]:
print("Testing accuracy: {:.2f}%".format(test_acc * 100))

Testing accuracy: 13.09%


In [6]:
import numpy as np
# Load test set
test_df = pd.read_csv('final_dataset.csv')

# Create data generator for test set
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=None,
    x_col="local_filename",
    y_col="label",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Predict labels for test set
y_pred = model.predict(test_generator)
predicted_labels = np.argmax(y_pred, axis=1)
actual_labels = test_generator.classes

# Map label indices to label names
label_map = {v: k for k, v in test_generator.class_indices.items()}
predicted_labels = [label_map[label] for label in predicted_labels]
actual_labels = [label_map[label] for label in actual_labels]

# Get fitzpatrick_scale values for test set
fitzpatrick_scale = test_df['fitzpatrick_scale']

# Combine the predicted labels, actual labels, and fitzpatrick_scale values into a DataFrame
results_df = pd.DataFrame({'predicted_label': predicted_labels, 'actual_label': actual_labels, 'fitzpatrick_scale': fitzpatrick_scale})

# Save the DataFrame to a CSV file
results_df.to_csv('results_InceptionNet_tr1n2.csv', index=False)

# Print individual accuracies for each fitzpatrick scale value
for scale in range(3, 7):
    scale_df = results_df[results_df['fitzpatrick_scale'] == scale]
    total = len(scale_df)
    correct = sum(scale_df['predicted_label'] == scale_df['actual_label'])
    accuracy = correct / total
    print(f"Fitzpatrick Scale {scale}: {accuracy:.2%} ({correct}/{total})")


Found 12215 validated image filenames belonging to 114 classes.
382/382 [==============================] - 1777s 5s/step
Fitzpatrick Scale 3: 1.26% (29/2295)
Fitzpatrick Scale 4: 1.36% (23/1689)
Fitzpatrick Scale 5: 0.67% (6/891)
Fitzpatrick Scale 6: 1.17% (5/427)


In [7]:
#UPDATED CODE
import pandas as pd

# read csv file into a pandas dataframe
df = pd.read_csv("results_InceptionNet_tr1n2.csv")

# compute total count of fitzpatrick_scale
total_count = df.shape[0]

print(total_count)

# compute count of correctly predicted values
correctly_predicted = df[df["actual_label"] == df["predicted_label"]].shape[0]
print(correctly_predicted)

answer = (correctly_predicted/total_count)*100

print(answer)

12215
182
1.489971346704871


In [9]:
accuracy = 13.09
d_accuracy = 1.49
new = accuracy/d_accuracy

f=[1.26,1.36,0.67,1.17]
l = [None] * 4

for i in range(0,4):
    l[i]=f[i]*new
print(l)

[11.069395973154363, 11.94791946308725, 5.886107382550336, 10.278724832214765]
